In [1]:
from ETGEMs_function_PMI import *
import pandas as pd
import cobra
import ast
from cobra.io import write_sbml_model
from numpy import *
import copy
import math

In [2]:
#Get Reaction G0 from local file_original
reaction_g0_file_original0 = './reaction_g0_strain0.txt'
reaction_g0_file_original1 = './reaction_g0_strain1.txt'
#Get Metabolite concentration from local file
metabolites_lnC_file_original0 = './metabolites_lnC_strain0.txt'
metabolites_lnC_file_original1 = './metabolites_lnC_strain1.txt'
#Get Model from local file
model_file_original0 = './iML1515.xml'
model_file_original1 = './iMM904.xml'
#Get reaction kcat data from ECMpy
reaction_kcat_MW_file_original0 = './ID_kcat_MW_file_strain0.csv'
reaction_kcat_MW_file_original1 = './ID_kcat_MW_file_strain1.csv'

In [3]:
## Convert to usable model formats
model0=Get_Concretemodel_Need_Data(reaction_g0_file_original0,metabolites_lnC_file_original0,model_file_original0,reaction_kcat_MW_file_original0)
model1=Get_Concretemodel_Need_Data(reaction_g0_file_original1,metabolites_lnC_file_original1,model_file_original1,reaction_kcat_MW_file_original1)

In [4]:
# get the information of km, vmax and public metabolites
km = pd.read_csv('./km.csv')
vmax = pd.read_csv('./vmax.csv')
public_metabolism = pd.read_csv('./public_metabolism_test.csv')

'''
public_metabolism_name_list = []
public_metabolism_concentration_list = []
for i in public_metabolism['metabolism']:
    public_metabolism_name_list.append(i)
for j in public_metabolism['concentration']:
    public_metabolism_concentration_list.append(j)
public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
print(public_metabolism)
'''

"\npublic_metabolism_name_list = []\npublic_metabolism_concentration_list = []\nfor i in public_metabolism['metabolism']:\n    public_metabolism_name_list.append(i)\nfor j in public_metabolism['concentration']:\n    public_metabolism_concentration_list.append(j)\npublic_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))\nprint(public_metabolism)\n"

In [5]:
# definate the function to simulate metabolism and interaction
def time_space_state(model_list, biomass_list, growth_list, breed_list, parameter_list, public_metabolism, vmax, km, deta_t, deta_x, micro_distribute_threshold, length, D, o2_concentration):
    
    number_cell_side = length // deta_x
    number_cell_side = int(number_cell_side)
    public_metabolism_name_list = []
    public_metabolism_concentration_list = []
    for i in public_metabolism['metabolism']:
        public_metabolism_name_list.append(i)
    for j in public_metabolism['concentration']:
        public_metabolism_concentration_list.append(j)
    public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
    
    number_model = len(model_list)
    
    k_m = {}
    v_max = {}
    for i in range(number_model):
        for j in range(len(km['reactions_strain'+str(i)])):
            k_m[(i, km['reactions_strain'+str(i)][j])] = km['km_strain'+str(i)][j]
            v_max[(i, vmax['reactions_strain'+str(i)][j])] = vmax['vmax_strain'+str(i)][j]
    v_max_o2 = v_max[(0,'EX_glc__D_e_reverse')]
    k_m_o2 = k_m[(0,'EX_glc__D_e_reverse')]        
    
    number_public_metabolism = len(public_metabolism)
    distribute_micro_list = {}
    distribute_public_metabolism_list = {}
    distribute_lb_list = {}
    public_metabolism_r_list = []
    #set the initial distribution of straints
    for i in range(number_model):
        distribute_micro = np.random.randint(20, size=number_cell_side)
        distribute_micro_list.update({i: distribute_micro+1})
    print(distribute_micro_list)
    distribute_micro_list_initial = copy.deepcopy(distribute_micro_list)
    #set the initial distribution of metabolites
    for i in public_metabolism_name_list:
        distribute_public_metabolism = multiply(np.mat(np.ones(number_cell_side)), public_metabolism[i])
        distribute_public_metabolism_list.update({i: distribute_public_metabolism})
    print("distribute_public_metabolite_initial: ", distribute_public_metabolism_list['glc__D_e'][0::])
    #calculate the upperbounds of uptake reactions for public metabolites
    public_reaction_ub_list = {}
    public_reaction_list = {}
    for i in range(number_model):
        public_reaction_ub = {}
        public_reaction = []
        for rea in model_list[i]['reaction_list']:
            if 'EX_' in rea:
                for j in public_metabolism_name_list:
                    try:
                        model_list[i]['coef_matrix'][(j,rea)]
                    except:
                        pass
                    else:
                        ub = np.mat(np.ones(number_cell_side))
                        if model_list[i]['coef_matrix'][(j,rea)] > 0:
                            for m in range(number_cell_side):
                                ub[0,m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                        else:
                            try:
                                model_list[i]['ub_list'][rea]
                            except:
                                ub = ub * 1000
                            else:
                                ub = ub * model_list[i]['ub_list'][rea]
                        public_reaction_ub.update({rea: ub})
                        public_reaction.append(rea)
                        break          
            public_reaction_ub_list[i] = public_reaction_ub
            public_reaction_list[i] = public_reaction

            
    ct = 0
    
    distribute_micro_list_t = {ct: distribute_micro_list}
    distribute_public_metabolism_list_t = {ct: distribute_public_metabolism_list}
    distribute_lb_list_t = {ct: distribute_lb_list}
    r = micro_distribute_threshold + 1
    r_threshold = [r]*5
    slip_r = np.mean(r_threshold[-5:])
    number = {}
    various = {}
    for i in range(number_model):
        number[i] = [np.mean(distribute_micro_list[i])]
        various[i] = [np.std(distribute_micro_list[i])/np.mean(distribute_micro_list[i])]
    
            
    # iterative simulation by slip_r
    biomass = {}
    while slip_r >= micro_distribute_threshold:
        ct += 1
        print(ct)
        distribute_micro_list_t[ct] = copy.deepcopy(distribute_micro_list_t[ct-1])
        micro_decrease = {}
        micro_increase = {}
        
        #simulate the cell wandering
        #micro_decrease: the decrease number of strains
        #micro_increase: the increase number of strains
        #micro_decrease_cell: the decrease number of strains in the current grid
        #micro_increase_fcell: the increase number of strains in the forward grid
        #micro_increase_bcell: the increase number of strains in the backward grid
        for m in range(number_cell_side):
            micro_decrease_cell = {}
            micro_increase_cell = {}
            for i in range(number_model):
                micro_decrease_cell[i] = 0
                micro_increase_cell[i] = 0
            micro_decrease[m] = micro_decrease_cell
            micro_increase[m] = micro_increase_cell
            
            
        if ct > 0:
            met = 'glc__D_e'
            threshold = 0.3
            for m in range(number_cell_side):
                #calculate the number of strains in the internal grids
                if 0<m<number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001)*deta_t/(1/15))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = (distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)*deta_t/(1/15))
                                    micro_increase_fcell = micro_increase_fcell_ratio * micro_decrease_cell
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_increase_bcell = micro_decrease_cell-micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                                    micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                                else:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001)*deta_t/(1/15))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001)*deta_t/(1/15))
                                    micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                            elif distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001)*deta_t/(1/15))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001)*deta_t/(1/15))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the first grid
                elif m == 0:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001)*deta_t/(1/15))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001)*deta_t/(1/15))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the last grid
                elif m == number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001)*deta_t/(1/15))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001)*deta_t/(1/15))
                                micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_fcell = micro_increase_fcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
        
            #calculate the number of strains after wandering
            for m in range(number_cell_side):
                for i in range(number_model):
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] - micro_decrease[m][i]
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] + micro_increase[m][i]
                    distribute_micro_list_t[ct][i][m] = max(0, distribute_micro_list_t[ct][i][m])
                            
                            
        
        #simulate the substrate diffusion by Fick's second law
        for m in range(number_cell_side): 
            if 0<m<number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + ((distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x-(distribute_public_metabolism_list[met][0,m]-distribute_public_metabolism_list[met][0,m-1])/deta_x)/deta_x*D*deta_t
                    if distribute_public_metabolism_list[met][0,m] < 0:
                        print('Warning: the D is too high!')
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == 0:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m-1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
        
        #simulate the metabolism by ETMs
        biomass_value_list = {}
        number_model_range = []
        biomass_strain = {}
        for m in range(number_cell_side):
            biomass_strain[m] = [0, 0]
            if m%2 == 0:
                number_model_range.append([0,1])
            elif m%2 == 1:
                number_model_range.append([1,0])
            
            
            biomass_value_micro = {}
            for i in number_model_range[m]:
                if distribute_micro_list_t[ct][i][m] > 0:
                    public_metabolism_flux_list = {}
                    for j in public_metabolism_name_list:
                        public_metabolism_flux_list.update({j: 0})
                    for j in public_reaction_list[i]:
                        model_list[i]['ub_list'][j] = public_reaction_ub_list[i][j][0,m]
                    model_list[i]['ub_list']['EX_o2_e_reverse'] = min(v_max_o2*o2_concentration/(o2_concentration+k_m_o2), model_list[i]['ub_list']['EX_o2_e_reverse'])   
            
                    biomass_id = biomass_list[i]
                    E_total=parameter_list[i][0]
                    #set the carbon source to glucose
                    substrate_name='EX_glc__D_e_reverse'
                    substrate_value=parameter_list[i][1]
                    biomass_value=growth_list[i]
                    K_value=parameter_list[i][2]

                    #calculate the MDF values of metabolic networks
                    biomass_value_micro[i] = biomass_value
                    obj_name=biomass_list[i]
                    obj_target='maximize'
                    if i == 0:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation0(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation1(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    biomass_value=max_biomass_under_mdf*0.9
                        
                    #calculate the minimum value of the sum of the fluxes
                    if i == 0:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation0(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation1(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
            
                    #translating the results of ETMs into a usable form
                    model=model_list[i]['model']
                    reaction_kcat_MW=model_list[i]['reaction_kcat_MW']
                    reaction_g0=model_list[i]['reaction_g0']
                    coef_matrix=model_list[i]['coef_matrix']
                    metabolite_list=model_list[i]['metabolite_list']
                    use_result = Get_Results_Thermodynamics(model,Concretemodel,reaction_kcat_MW,reaction_g0,coef_matrix,metabolite_list)
            
                    if distribute_micro_list_t[ct][i][m] == 0:
                        biomass_strain[m][i] = 0
                    else:
                        biomass_strain[m][i] = use_result['flux'][biomass_list[i]]
                
                    #simulate the fluxes of the public metabolites
                    for rea in public_reaction_list[i]:
                        for met in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(met,rea)]  
                            except:
                                pass
                            else:
                                public_metabolism_flux_list[met] = public_metabolism_flux_list[met] + model_list[i]['coef_matrix'][(met,rea)]* use_result['flux'][rea]
                        
                    #simulate the distribution of the public metabolites
                    distribute_public_metabolism_ori = copy.deepcopy(distribute_public_metabolism_list)
                    distribute_public_metabolism_re = copy.deepcopy(distribute_public_metabolism_ori)
                    for met in public_metabolism_name_list:
                        distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] - public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m]
                        distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                        if distribute_public_metabolism_list[met][0,m] < 0:
                            print(met+'_1: ', distribute_public_metabolism_list[met][0,m])
                                
                    #simulate the multiplication and death rates of strains and the distribution of public metabolites after multiplication or death        
                    death_rate = 0
                    birth_rate = 1
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                bd_rate_met = (distribute_public_metabolism_list[met][0,m] - 0.1)/intracellular_c_dict[met]/distribute_micro_list_t[ct][i][m]
                                if bd_rate_met < 0:
                                    death_rate_lb = (public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] - distribute_public_metabolism_re[met][0,m] + 0.1)/(public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] + intracellular_c_dict[met] * distribute_micro_list_t[ct][i][m])
                                    death_rate = max(death_rate, death_rate_lb)
                                else:
                                    birth_rate = min(birth_rate, bd_rate_met)
                    death_rate = min(death_rate, 1)
                    birth_rate = min(birth_rate,1)
                    if death_rate > 0:
                        birth_rate = 0
                        for met in public_metabolism_name_list:
                            distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + public_metabolism_flux_list[met] * deta_t * math.ceil(distribute_micro_list_t[ct][i][m] * death_rate)
                            distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                    
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_ori[met][0,m] - intracellular_c_dict[met]*math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i]) + intracellular_c_dict[met]*math.ceil(death_rate*distribute_micro_list_t[ct][i][m])
                                if distribute_public_metabolism_list[met][0,m] < 0:
                                    print(met+'_2: ', distribute_public_metabolism_list[met][0,m])
                                    distribute_public_metabolism_list[met][0,m] = 0
                        
                    #simulate the distribution of strains after multiplication or death
                    distribute_micro_increase = math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i])
                    distribute_micro_decrease = math.ceil(distribute_micro_list_t[ct][i][m]*death_rate)
                    distribute_micro_list_t[ct][i][m] = max(0,distribute_micro_list_t[ct][i][m] + distribute_micro_increase - distribute_micro_decrease)
            
            #calculate the upperbounds of nutrient uptake rates after substrate diffusion, cell wandering, metabolism, multiplication and death
            biomass_value_list[m] = biomass_value_micro
            
            for i in range(number_model):
                for rea in model_list[i]['reaction_list']:
                    if 'EX_' in rea:
                        for j in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(j,rea)]
                            except:
                                pass
                            else:
                                if model_list[i]['coef_matrix'][(j,rea)] > 0:
                                    public_reaction_ub_list[i][rea][0, m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                   
        
        if ct == 1:
            biomass_strain_initial = copy.deepcopy(biomass_strain)
            
        print("distribute_public_metabolite: ", distribute_public_metabolism_list['glc__D_e'][0::])
        # calculate the mean number and uniformity of distribution of strains at this iteration
        for i in range(number_model):
            strain_number = np.mean(distribute_micro_list_t[ct][i])
            strain_various = np.std(distribute_micro_list_t[ct][i])
            number[i].append(strain_number)
            various[i].append(strain_various)
            print('strain_number: ', i, strain_number)
            print('strain_various: ', i, strain_various)
        
        # calculate the slip_r at this iteration
        if ct > 1:
            r = 0
            for i in range(number_model):
                for m in range(number_cell_side):
                    if distribute_micro_list_t[ct-1][i][m] > 0:
                        r = r + abs((distribute_micro_list_t[ct][i][m]-distribute_micro_list_t[ct-1][i][m])/(distribute_micro_list_t[ct-1][i][m])/deta_t)
                    else:
                        r = r + abs(distribute_micro_list_t[ct][i][m]/deta_t)
       
        r_threshold.append(r)
        slip_r = np.mean(r_threshold[-1:])
        fd_r_threshold = r_threshold[1:]
        print('fd_r_threshold: ', fd_r_threshold)
        print('slip_r: ', slip_r)
        
        biomass_x = {}
        biomass_x_initial = {}
        for i in range(number_model):
            biomass_x[i] = [1]*number_cell_side
            if ct == 1:
                biomass_x_initial[i] = np.ones(number_cell_side)
            for m in range(number_cell_side):
                biomass_x[i][m] = distribute_micro_list_t[ct][i][m]*biomass_strain[m][i]
                if ct == 1:
                    biomass_x_initial[i][m] = distribute_micro_list_initial[i][m]*biomass_strain_initial[m][i]
        biomass[ct] = biomass_x
        
        if ct == 1:
            print('_1: ', distribute_micro_list_initial)
            print('_2: ', biomass_strain_initial)
            print('_3: ', biomass_x_initial)
            
    return(biomass_x_initial, biomass, number, various)

In [6]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 0.5, 0.1, 30, 0.02, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 8, 12, 17, 10, 18,  8,  5, 20, 20, 11,  1,  8,  7, 16,  5,  7, 13,
       14, 17,  7, 14,  1,  7,  3,  3,  6,  6,  3, 12, 14, 17, 13, 20,  1,
        8, 20,  2, 19, 19, 17,  8, 19, 17, 14, 18, 19,  1,  2,  9,  3,  2,
       11, 14, 15, 12, 20,  3, 17,  6, 18]), 1: array([14,  1, 15, 20, 11, 19, 15, 20, 11, 12,  9, 19, 11, 15,  5,  2, 16,
        8,  1, 11, 15, 20, 18,  1,  6, 18, 20, 13, 19, 12,  3, 10,  1, 20,
       19,  5,  6,  2, 18, 19, 15,  2, 10,  6, 17,  5,  6,  6,  4, 17, 17,
       10, 11, 19, 18,  5, 14,  7, 17, 12])}
distribute_public_metabolite_initial:  [[300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300.
  300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300.
  300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300.
  300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300.
  300. 300. 300. 300.]]
1
distribute_public_metabolite:  [[285.47578962 291.08716541 278.12399517 279.42305191 280.27834465
  

distribute_public_metabolite:  [[238.25917112 239.32569648 225.76544185 218.57083616 220.65153025
  228.47334799 214.62827061 210.3797594  220.41024879 231.71337664
  242.21306288 237.66823887 241.76462982 232.31824254 255.64416728
  250.52177286 237.8604279  243.88122029 239.83264598 240.95906294
  233.54365419 245.95290931 248.79532961 270.29578806 266.81495578
  255.5103411  244.93190099 242.86813932 232.17798234 227.29564227
  235.59864784 228.25108439 237.94658527 234.82449547 236.89662604
  231.81959433 242.03859373 235.74765139 218.35670304 211.55271347
  219.04411732 224.05621249 220.7290821  221.40657056 224.24291032
  243.97669314 264.32918952 272.22272605 269.04667417 255.90625381
  250.83039535 236.36690053 231.04425925 220.94070909 216.9687762
  229.13154047 234.70129422 229.52047518 231.19304896 222.51466259]]
strain_number:  0 17.566666666666666
strain_various:  0 7.017517763109371
strain_number:  1 9.583333333333334
strain_various:  1 3.153261098538393
fd_r_threshold:  

glc__D_e_1:  -3.5935051812991574
glc__D_e_1:  -1.0864104825872865
glc__D_e_1:  -0.0064151007282085
glc__D_e_1:  -3.8354607760576744
glc__D_e_1:  -1.1988510216227226
glc__D_e_1:  -17.290712839893082
glc__D_e_1:  -4.566995543088332
glc__D_e_1:  -12.90459460477652
glc__D_e_1:  -15.784568847009
glc__D_e_1:  -0.39810311043721214
glc__D_e_1:  -10.693955368696166
glc__D_e_1:  -4.615598548172418
glc__D_e_1:  -2.8570918541210126
glc__D_e_1:  -5.095771282433212
glc__D_e_1:  -5.0324508698033315
distribute_public_metabolite:  [[1.75506988e+00 3.84549318e+00 3.18040657e+00 4.33936230e-01
  2.93845098e+00 6.84863996e-01 2.66924039e+00 3.48931896e+00
  1.34683508e+01 2.42498863e+01 1.63177424e+01 3.51733430e+01
  4.74839105e+01 3.22566257e+01 6.43940146e+01 5.25365307e+01
  5.06083468e+01 6.43486426e+01 6.63921182e+01 7.41337976e+01
  9.70566739e+01 1.27330261e+02 1.53064455e+02 1.80877820e+02
  1.80544555e+02 1.66853956e+02 1.42786704e+02 1.17167153e+02
  7.65036817e+01 4.20988220e+01 3.60817137e+01

glc__D_e_1:  -17.485430743128475
glc__D_e_1:  -1.9159663912910894
glc__D_e_1:  -1.1986775685722688
glc__D_e_1:  -7.177962510031858
glc__D_e_1:  -13.06206285230258
glc__D_e_1:  -1.3849645884245394
glc__D_e_1:  -3.7133433309608184
glc__D_e_1:  -19.325715919300137
glc__D_e_1:  -1.5672308068996195
glc__D_e_1:  -6.433191831668861
glc__D_e_1:  -4.429399503792881
glc__D_e_1:  -6.069133420437203
glc__D_e_1:  -22.897707602215434
glc__D_e_1:  -1.2468564018555703
glc__D_e_1:  -7.374814789835375
glc__D_e_1:  -13.976448846010214
glc__D_e_1:  -0.11032982156904403
glc__D_e_1:  -0.7199758719504118
glc__D_e_1:  -10.862034510454954
glc__D_e_1:  -15.615291389053334
glc__D_e_1:  -0.22882565248154663
glc__D_e_1:  -1.4341210553621748
glc__D_e_1:  -33.43074160335272
glc__D_e_1:  -18.39061790044629
glc__D_e_1:  -0.20543543267894715
glc__D_e_1:  -22.835815162318347
glc__D_e_1:  -28.597039939579865
glc__D_e_1:  -0.3791264575306137
glc__D_e_1:  -14.915328749008582
glc__D_e_1:  -17.03516822056543
glc__D_e_1:  -9.

glc__D_e_1:  -28.560990618369704
glc__D_e_1:  -18.00519577753062
glc__D_e_1:  -38.762672601939066
glc__D_e_1:  -12.994076815146498
glc__D_e_1:  -7.808921794875227
glc__D_e_1:  -3.195175405355462
glc__D_e_1:  -8.385331663994842
glc__D_e_1:  -2.1681105163362138
glc__D_e_1:  -6.477223373103711
glc__D_e_1:  -3.898493009723812
glc__D_e_1:  -2.6455658236814545
glc__D_e_1:  -0.0026416011449315047
glc__D_e_1:  -1.8191528455595671
distribute_public_metabolite:  [[ 2.79690568  1.29967777  0.85094272  0.19901574  0.68567109  1.45144846
   1.01790884  1.2571836   2.89172961  1.58525705  1.29678773  1.0023657
   0.86535478  1.13908989  0.51571782  0.55613427  1.57054699  0.81784375
   0.34843895  1.16224391  0.89359489  1.10318683  7.1765669  20.30396592
  30.14141414 15.70795976  0.48921765  1.04725306  1.161415    1.23345267
   0.24116465  1.25540322  0.39034002  1.11908092  1.1362752   0.9351126
   1.59506357  1.01881034  0.75318797  1.61727127  1.524964    0.8017349
   0.89058213  0.13185556  1

glc__D_e_1:  -2.321527792053841
glc__D_e_1:  -0.8786332716315782
glc__D_e_1:  -1.733129617168648
glc__D_e_1:  -1.3578541042928634
glc__D_e_1:  -2.633588356238074
glc__D_e_1:  -1.4469342305284543
glc__D_e_1:  -3.9728221988336414
glc__D_e_1:  -4.285849918199665
glc__D_e_1:  -1.8239291882365882
glc__D_e_1:  -10.367675390309257
glc__D_e_1:  -18.49689440742134
glc__D_e_1:  -32.53322570591825
glc__D_e_1:  -14.214269889633613
glc__D_e_1:  -10.289461518973548
glc__D_e_1:  -6.085532653245667
glc__D_e_1:  -1.291122746651335
glc__D_e_1:  -0.4870081062562541
glc__D_e_1:  -1.041781415524699
glc__D_e_1:  -0.0388562995861666
glc__D_e_1:  -0.6361522997128446
glc__D_e_1:  -0.16816360940406916
glc__D_e_1:  -0.10203367927086371
glc__D_e_1:  -0.3216793718593245
glc__D_e_1:  -0.23042386606744508
glc__D_e_1:  -0.37539628994468344
glc__D_e_1:  -0.4365800666896106
glc__D_e_1:  -3.191423524197331
glc__D_e_1:  -3.702058019966207
glc__D_e_1:  -9.2223074695882
glc__D_e_1:  -3.303808717538318
glc__D_e_1:  -7.09006

glc__D_e_1:  -0.06334428975009437
glc__D_e_1:  -1.5969361375799271
glc__D_e_1:  -0.5865119613480068
glc__D_e_1:  -0.6202198561676724
glc__D_e_1:  -0.9963594005511429
glc__D_e_1:  -0.39536222328881
glc__D_e_1:  -0.3069112274553508
distribute_public_metabolite:  [[1.56499572 1.15878481 1.20152914 0.66361248 1.02838002 1.32724444
  1.38737763 0.19746946 1.40451066 0.26484653 0.16662638 1.20824155
  0.63379852 0.16939293 0.599693   0.17309331 0.65159317 0.8869127
  0.47205926 1.45700242 1.20779377 0.99629572 0.56893114 1.44375729
  0.93383475 0.90012686 0.52398731 1.12498449 0.73199875 1.45562061
  0.95917592 0.47621477 0.3053612  0.94228709 0.84599906 1.25493482
  1.33820726 0.90819731 1.1912795  1.01605523 0.91257515 0.77884829
  1.27836004 1.4917097  1.14784376 1.51882791 1.20282733 0.74681479
  1.21343548 0.57319183 1.40209817 0.47671295 1.37827299 1.08847894
  1.4822854  0.72696154 0.95151698 1.06512381 1.3538148  1.01610387]]
strain_number:  0 0.2833333333333333
strain_various:  0 0.